In [ ]:
%pip install "calcbench-api-client[Pandas]"

In [ ]:
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pandas as pd
import calcbench as cb
from scipy.stats import zscore
from datetime import date
from qgridnext import show_grid

### File is the Calcbench Point-In-Time data, see https://github.com/calcbench/notebooks/blob/master/standardized_numeric_point_in_time.ipynb

### Talk to us@calcbench.com if you want a historical file.


In [ ]:
# last date
last_date = pd.to_datetime(date(2024, 9, 30))

# XBRL data starts ~2010
first_date = pd.to_datetime(date(2010, 1, 1))

In [ ]:
d = pq.read_table("C:/Users/andre/Downloads/standardized_data_oct_23.parquet")

In [ ]:
# Filter for the line items we want

expenses = [
    "CostOfRevenue",
    "SGAExpense",
    "InterestExpense",
    "IncomeTaxes",
]
expr = pc.field("metric").isin(["Revenue", "NetIncome"] + expenses)
metric_data = d.filter(expr).to_pandas()

In [ ]:
# do things in millions for readability

metric_data["value"] = metric_data["value"].astype(float) / 1e6

In [ ]:
# Remove small companies

# revenue = revenue[revenue["value"] > 1e7]

In [ ]:
# Quarterly only
metric_data = metric_data[
    ~metric_data.index.get_level_values("fiscal_period").str.endswith("-0")
]

## Add SIC Code meta-data


In [ ]:
company_data = cb.companies(entire_universe=True)

In [ ]:
merged = metric_data.reset_index().merge(
    company_data[["ticker", "sic_code"]], right_on=["ticker"], left_on=["ticker"]
)[["ticker", "period_end", "sic_code", "value", "metric"]]

##### Split SIC code into levels, GICs codes would probably be better but Calcbench does not have them

###### https://www.sec.gov/corpfin/division-of-corporation-finance-standard-industrial-classification-sic-code-list


In [ ]:
merged["top_level_SIC_code"] = merged["sic_code"].floordiv(1000).astype("string")
merged["second_level_SIC_code"] = (
    (merged["sic_code"] % 1000).floordiv(100).astype("string")
)
merged["third_level_SIC_code"] = (
    (merged["sic_code"] % 100).floordiv(10).astype("string")
)
merged["fourth_level_SIC_code"] = (merged["sic_code"] % 10).astype("string")

In [ ]:
merged["ds"] = (merged["period_end"] + pd.offsets.QuarterEnd()).dt.normalize()

In [ ]:
merged = merged[
    ~merged.isnull().T.any()
]  # get rid of rows with any null values, the model builder does not like nulls

In [ ]:
# Get first record, there are revisions in the Calcbench data set.
merged = merged.groupby(["ticker", "ds", "metric"]).first()

In [ ]:
# only want companies for which we have more than a year of history
merged = merged.groupby("ticker").filter(lambda g: g.shape[0] > 4)

In [ ]:
# throw out outliers
z_scores = merged.groupby(["ticker", "metric"])["value"].transform(lambda x: zscore(x))
merged = merged[z_scores < 4]

In [ ]:
accounts = (
    merged.reset_index()
    .set_index(
        [
            "top_level_SIC_code",
            "second_level_SIC_code",
            "third_level_SIC_code",
            "fourth_level_SIC_code",
            "ticker",
            "ds",
            "metric",
        ]
    )["value"]
    .unstack("metric")
)

In [ ]:
calculated_net_income = accounts["Revenue"] - accounts[expenses].sum(axis=1)

In [ ]:
accounts["Other"] = accounts["NetIncome"] - calculated_net_income

In [ ]:
# No longer need NetIncome
accounts = accounts.drop('NetIncome', axis=1)

In [ ]:
merged = accounts.stack("metric").to_frame(name='y')

In [ ]:
merged = merged.reset_index('ds')

In [ ]:
merged = merged[merged["ds"] <= last_date]
merged = merged[merged["ds"] >= first_date]

In [ ]:
# remove companies for which we do not have current data
merged = merged.groupby("ticker").filter(lambda x: x["ds"].max() == last_date)

In [ ]:
# Get rid of companies for which we do not have enough data
MINIMUM_NUMBER_OF_OBSERVATIONS = 40
merged = merged.groupby("ticker").filter(
    lambda x: x.shape[0] > MINIMUM_NUMBER_OF_OBSERVATIONS
)

In [ ]:
merged = merged.reset_index()

In [ ]:
pivot_columns = [
    "ticker",
    "top_level_SIC_code",
    "second_level_SIC_code",
    "third_level_SIC_code",
    "fourth_level_SIC_code",
    "metric",
]

# forward fill missing data
merged = (
    merged.pivot(
        index="ds",
        columns=pivot_columns,
        values="y",
    )
    .ffill()
    .melt(ignore_index=False, value_name="y")
    .reset_index()
)

In [ ]:
merged = merged[~merged["y"].isna()]

In [ ]:
# top level
merged["market"] = "market"

In [ ]:
merged

In [ ]:
merged.to_parquet("test_train_data.parquet", index=False)